# -*- coding: utf-8 -*-

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from datetime import datetime, timedelta
from scipy.integrate import odeint
from sklearn.preprocessing import StandardScaler
import ipywidgets as widgets
from IPython.display import display
import json
import os

# Для ARIMA

In [5]:
from statsmodels.tsa.arima.model import ARIMA

# ---------------------------
# НАСТРОЙКА
# ---------------------------

In [8]:
sns.set(style="whitegrid")
pd.options.mode.chained_assignment = None  # Отключаем предупреждения о копировании
plt.rcParams['figure.figsize'] = (12, 6)

# ---------------------------
# ЗАГРУЗКА И ПРЕДОБРАБОТКА ДАННЫХ
# ---------------------------

In [11]:
# Загрузка набора данных
data_final = pd.read_csv("dataset_final.csv", parse_dates=["Date"])

In [13]:
# Очистка данных
cols_numeriques = ['new_cases', 'new_vaccinations', 'people_fully_vaccinated']
for col in cols_numeriques:
    data_final[col] = pd.to_numeric(data_final[col], errors='coerce').fillna(0)

In [15]:
# Преобразование категориальных столбцов
data_final['ID_Localisation'] = data_final['ID_Localisation'].astype('category')
data_final['location'] = data_final['location'].astype('category')
data_final['continent'] = data_final['continent'].astype('category')

In [17]:
# Сортировка данных по коду страны и дате
data_final.sort_values(by=["ID_Localisation", "Date"], inplace=True)

# ---------------------------
# ВЫЧИСЛЕНИЕ МЕТРИК
# ---------------------------

In [20]:
def calcul_metriques(df):
    """Вычисляет ключевые метрики для данного DataFrame с использованием
       населения для расчёта уровня вакцинации."""
    # Агрегация по странам
    grouped = df.groupby('ID_Localisation').agg(
        Cas_Total=('new_cases', 'sum'),
        Vaccinations_Total=('new_vaccinations', 'sum'),
        Dernier_Type_Mesure=('Type_Mesure', 'last'),
        Population=('population', 'max'),
        Vaccinated=('people_fully_vaccinated', 'max')
    ).reset_index()
    
    # Расчёт максимального уровня вакцинации (в процентах)
    grouped['Couverture_Vaccinale_Max'] = (grouped['Vaccinated'] / grouped['Population']) * 100
    
    # Расчёт среднего темпа роста новых случаев для каждой страны
    taux_croissance = df.groupby('ID_Localisation')['new_cases'].pct_change().groupby(df['ID_Localisation']).mean().reset_index()
    taux_croissance.columns = ['ID_Localisation', 'Taux_Croissance_Moyen']
    
    # Объединение результатов
    metriques = pd.merge(grouped, taux_croissance, on='ID_Localisation', how='left')
    
    # Преобразование типов:
    # Числовые значения в целые числа и уровень вакцинации округляем до 2 знаков после запятой
    metriques['Cas_Total'] = metriques['Cas_Total'].astype(int)
    metriques['Vaccinations_Total'] = metriques['Vaccinations_Total'].astype(int)
    metriques['Population'] = metriques['Population'].astype(int)
    metriques['Couverture_Vaccinale_Max'] = metriques['Couverture_Vaccinale_Max'].round(2).astype(float)
    
    return metriques

# Метрики по странам
metrics_pays = calcul_metriques(data_final)

# Глобальные метрики (для всех стран)
metrics_global = pd.DataFrame([{
    'ID_Localisation': 'GLOBAL',
    'Taux_Croissance_Moyen': data_final['new_cases'].pct_change().mean(),
    'Couverture_Vaccinale_Max': (data_final['people_fully_vaccinated'].max() / data_final['population'].max() * 100),
    'Cas_Total': int(data_final['new_cases'].sum()),
    'Vaccinations_Total': int(data_final['new_vaccinations'].sum()),
    'Dernier_Type_Mesure': data_final['Type_Mesure'].last_valid_index()
}])

# Объединение метрик
metrics_complet = pd.concat([metrics_pays, metrics_global], ignore_index=True)


C:\Users\manas\AppData\Local\Temp\ipykernel_15772\4261115101.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = df.groupby('ID_Localisation').agg(
C:\Users\manas\AppData\Local\Temp\ipykernel_15772\4261115101.py:17: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  taux_croissance = df.groupby('ID_Localisation')['new_cases'].pct_change().groupby(df['ID_Localisation']).mean().reset_index()


# ---------------------------
# КЛАСТЕРИЗАЦИЯ
# ---------------------------

In [23]:
# Присвоение кластера на основе Couverture_Vaccinale_Max
def assign_cluster(couv):
    if couv >= 70:
        return 1
    elif couv >= 30:  # От 30 до 69,9%
        return 2
    else:
        return 3

metrics_complet['Cluster'] = metrics_complet['Couverture_Vaccinale_Max'].apply(assign_cluster)


In [28]:
# Проверка на наличие NaN значений
print(metrics_complet['Population'].isna().sum())

# Обработка NaN значений
metrics_complet['Population'].fillna(0, inplace=True)  # или используйте другой метод

# Преобразование типов данных
metrics_complet['Cas_Total'] = metrics_complet['Cas_Total'].astype(int)
metrics_complet['Vaccinations_Total'] = metrics_complet['Vaccinations_Total'].astype(int)
metrics_complet['Population'] = metrics_complet['Population'].astype(int)
metrics_complet['Couverture_Vaccinale_Max'] = metrics_complet['Couverture_Vaccinale_Max'].round(2).astype(float)


1


# ---------------------------
# ВИЗУАЛИЗАЦИЯ (для предварительного просмотра)
# ---------------------------

In [30]:
# 1. Географическая карта (размер пузырьков пропорционален уровню вакцинации)
df_map = metrics_complet[metrics_complet['ID_Localisation'] != 'GLOBAL']
fig_map = px.scatter_geo(df_map,
                    locations="ID_Localisation",
                    color="Couverture_Vaccinale_Max",
                    size="Couverture_Vaccinale_Max",  # размер зависит от процента
                    hover_name="ID_Localisation",
                    title="Максимальный уровень вакцинации по странам")

In [32]:
# 2. График изменения во времени (случаи и вакцинации)
evolution_globale = data_final.groupby('Date').agg({
    'new_cases': 'sum',
    'new_vaccinations': 'sum'
}).reset_index()
fig_evol = px.line(evolution_globale, x='Date', y=['new_cases', 'new_vaccinations'],
              title="Глобальное изменение случаев и вакцинаций")

# ---------------------------
# МОДЕЛЬ SEIR ДЛЯ СТРАН (Ежедневное моделирование и месячная кумуляция)
# ---------------------------

In [35]:
def modele_seir(population, initial_infectes, jours=180, start_date=None):
    """Запускает модель SEIR для заданного населения и возвращает DataFrame с ежедневной симуляцией.
       start_date должен быть объектом datetime для построения временной оси."""
    beta = 0.4   # Коэффициент передачи
    sigma = 1/5  # Показатель инкубации
    gamma = 1/14 # Коэффициент выздоровления
    
    def deriv(y, t):
        S, E, I, R = y
        dSdt = -beta * S * I / population
        dEdt = beta * S * I / population - sigma * E
        dIdt = sigma * E - gamma * I
        dRdt = gamma * I
        return dSdt, dEdt, dIdt, dRdt
    
    # Интервал времени в днях
    t = np.linspace(0, jours-1, jours)
    y0 = population - initial_infectes, 0, initial_infectes, 0
    ret = odeint(deriv, y0, t)
    df_sim = pd.DataFrame({
        'Jour': t,
        'Susceptibles': ret[:,0],
        'Exposes': ret[:,1],
        'Infectes': ret[:,2],
        'Retablis': ret[:,3]
    })
    # Построение временной оси: начинаем с последней даты наблюдений для данной страны
    if start_date is None:
        start_date = datetime.today()
    df_sim['Date'] = df_sim['Jour'].apply(lambda x: start_date + timedelta(days=int(x)))
    return df_sim

# Применение модели SEIR для каждой страны и агрегация по месяцам (кумуляция ежедневных значений)
predictions_globales = []
for iso_code in data_final['ID_Localisation'].unique():
    pays_data = data_final[data_final['ID_Localisation'] == iso_code]
    if not pays_data.empty:
        pop = int(pays_data['population'].iloc[0])
        # Используем последнее зарегистрированное значение новых случаев как начальное условие
        dernier_cas = int(pays_data['new_cases'].iloc[-1])
        # Определяем start_date как последняя дата наблюдений для данной страны
        start_date = pays_data['Date'].iloc[-1]
        # Ежедневное моделирование на 180 дней
        predictions = modele_seir(pop, dernier_cas, jours=180, start_date=start_date)
        predictions['ID_Localisation'] = iso_code
        predictions['Nom_Pays'] = pays_data['location'].iloc[0]
        # Агрегация по месяцам: суммирование ежедневных значений для каждого компонента
        predictions['Mois'] = predictions['Date'].dt.to_period('M').dt.to_timestamp()
        predictions_mensuelles = predictions.groupby(['ID_Localisation', 'Nom_Pays', 'Mois']).agg({
            'Susceptibles': 'sum',
            'Exposes': 'sum',
            'Infectes': 'sum',
            'Retablis': 'sum'
        }).reset_index()
        predictions_globales.append(predictions_mensuelles)

if predictions_globales:
    df_seir = pd.concat(predictions_globales, ignore_index=True)
    # Округляем и преобразуем значения к целым числам, если это имеет смысл
    df_seir['Susceptibles'] = df_seir['Susceptibles'].round().astype(int)
    df_seir['Exposes'] = df_seir['Exposes'].round().astype(int)
    df_seir['Infectes'] = df_seir['Infectes'].round().astype(int)
    df_seir['Retablis'] = df_seir['Retablis'].round().astype(int)
    df_seir.to_csv("powerbi_data/seir_predictions.csv", index=False)
else:
    print("Не сгенерировано никаких прогнозов SEIR.")


# ---------------------------
# ПРОГНОЗ ВРЕМЕННЫХ РЯДОВ С ARIMA (Годовой прогноз)
# ---------------------------

In [51]:
arima_predictions = []
for iso in data_final['ID_Localisation'].unique():
    # Sélectionner et préparer les données pour le pays iso
    cas_local = data_final[data_final['ID_Localisation'] == iso][['Date', 'new_cases']].copy()
    cas_local.set_index('Date', inplace=True)
    # Éliminer les doublons en regroupant par date
    cas_local = cas_local.groupby(level=0).sum()
    # Mettre la série à une fréquence journalière
    cas_local = cas_local.asfreq('D').fillna(0)
    
    # Vérifier qu'il y a suffisamment d'observations (par exemple, au moins 10 jours de données)
    if len(cas_local) < 10:
        print(f"Insufficient data pour {iso}, prévision non effectuée.")
        continue

    try:
        # Ajustement du modèle ARIMA sur la série journalière
        model = ARIMA(cas_local['new_cases'], order=(1,1,1))
        model_fit = model.fit()
        steps = 30  # Prévoir 30 jours
        forecast = model_fit.forecast(steps=steps)
        # Construction de l'index des prévisions : à partir du jour suivant la dernière date observée
        new_index = pd.date_range(start=cas_local.index.max() + pd.Timedelta(days=1),
                                  periods=steps, freq='D')
        forecast.index = new_index
        
        pred_df = forecast.reset_index()
        pred_df.columns = ['Date', 'Predicted_New_Cases']
        pred_df['ID_Localisation'] = iso
        # Arrondir les valeurs prédites et convertir en entier
        pred_df['Predicted_New_Cases'] = pred_df['Predicted_New_Cases'].round().astype(int)
        arima_predictions.append(pred_df)
    except Exception as e:
        print(f"Ошибка ARIMA для {iso}: {e}")

if arima_predictions:
    df_arima = pd.concat(arima_predictions, ignore_index=True)
    df_arima.to_csv("powerbi_data/arima_predictions.csv", index=False)
else:
    print("Не сгенерировано никаких прогнозов ARIMA.")


Insufficient data pour ESH, prévision non effectuée.


C:\Users\manas\anaconda3\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

C:\Users\manas\anaconda3\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

C:\Users\manas\anaconda3\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

C:\Users\manas\anaconda3\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

C:\Users\manas\anaconda3\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

C:\Users\manas\anaconda3\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

C:\U

# ---------------------------
# ЭКСПОРТ ДЛЯ POWER BI
# ---------------------------

In [53]:
def exporter_pour_powerbi():
    """Экспортирует данные в формате CSV для Power BI с преобразованием
       типов столбцов для оптимизации отображения."""
    os.makedirs("powerbi_data", exist_ok=True)
    
    # Экспорт исходных временных данных
    data_final_export = data_final.copy()
    data_final_export['new_cases'] = data_final_export['new_cases'].astype(int)
    data_final_export['new_vaccinations'] = data_final_export['new_vaccinations'].astype(int)
    data_final_export.to_csv("powerbi_data/donnees_brutes.csv", index=False)
    
    # Экспорт метрик
    metrics_complet.to_csv("powerbi_data/metriques.csv", index=False)
    
    # Экспорт кластеризации (например, отдельный файл)
    metrics_complet[['ID_Localisation', 'Cas_Total', 'Vaccinations_Total', 'Population', 
                     'Couverture_Vaccinale_Max', 'Taux_Croissance_Moyen', 'Cluster']].to_csv("powerbi_data/clusters.csv", index=False)
    
    print("✅ Данные экспортированы в powerbi_data/")

# Вызов функции экспорта
exporter_pour_powerbi()

# Конец скрипта

✅ Данные экспортированы в powerbi_data/
